In [1]:
from ltp import LTP
from itertools import chain
from collections import defaultdict

def build_T2H(dep):
    '''
    根据依存树构建每个词的发射字典
    txt: 设备机房、电梯机房、水箱间、天线
    dep: [
        (1, 2, 'ATT'), (2, 0, 'HED'), (3, 5, 'WP'), 
        (4, 5, 'ATT'), (5, 2, 'COO'), (6, 7, 'WP'), 
        (7, 2, 'COO'), (8, 9, 'WP'), (9, 2, 'COO')
    ]
    return {
        2: {'ATT': [1], 'COO': [5, 7, 9]}, 
        0: {'HED': [2]}, 
        5: {'ATT': [4]}
    }
    '''
    dep_T2H = defaultdict(dict)
    for d in dep:
        # if d[2] in ['WP','LAD']: continue
        if d[1] in dep_T2H:
            dep_T2H[d[1]][d[2]] = dep_T2H[d[1]].get(d[2],[])+[d[0]]
        else:
            dep_T2H[d[1]] = {d[2]:[d[0]]}
    return dep_T2H
    
def find_smallest(dep_T2H, p):
    '''
    寻找当前中心语的覆盖的围
    由于不存在交叉的情况，所以只要往前找最小的即可
    return: 第一个词对应的位置
    '''
    if p==0: return 1
    p_out = list(dep_T2H.get(p,{}).values())
    p_out = list(chain(*p_out))
    if not p_out or min(p_out)>p: return p
    smallest = find_smallest(dep_T2H, min(p_out))
    return smallest

def find_biggest(dep_T2H, p):
    '''
    寻找当前中心语的覆盖范围
    由于不存在交叉的情况，所以只要往后找最大的即可
    return: 最后一个词对应的位置
    '''
    p_out = list(dep_T2H.get(p,{}).values())
    p_out = list(chain(*p_out))
    if not p_out or max(p_out)<p: return p
    biggest = find_biggest(dep_T2H, max(p_out))
    return biggest

ltp = LTP('LTP/small')


In [2]:
# 危险做名词成分时替换为危险性
'建筑无法设置泄压设施或泄压面积不符合要求时，建筑中存在可燃气体或纤维爆炸危险性的部位的建筑承重结构应满足抗爆要求。'
'地面特殊设施基地或地下特殊设施的地面附属设施，距易发生危险的建筑物、仓库、储罐、可燃物品和材料堆场等，应满足安全防护距离的要求。'
# “及其”需要特殊处理
'给水厂应对制水生产中的主要设施、设备制定和实施巡查维护保养制度，应对主要工艺运行情况及其运行中的动态技术参数，制定和实施质量控制点检验制度'

'给水厂应对制水生产中的主要设施、设备制定和实施巡查维护保养制度，应对主要工艺运行情况及其运行中的动态技术参数，制定和实施质量控制点检验制度'

In [42]:
######### 添加表述规范化模块 #########
'''
识别出来符合，替换为同义词满足
城乡给水工程建设规模的划分应符合表2.0.23的要求
标准定语是带“的”的，实体连续修饰实体需要拆分，中心语要是简单概念
标准贯入试验设备应符合表 5.0.4 的规定。
, '符合表': '满足表'

绿道连接线不应直接借道国道、省道等干线公路及快速路等道路；
'''
# 把模型分词错误的地方替换为可以正确分词的内容
replace_dic = {'压强':'压力', '借道':'取道'}

ltp.add_words(words=list(replace_dic.keys()), freq=2)

def deal_miss_word(txt, replace_dic):
    '''
    未登录词加入词典后虽然可以获得正确的分词结果，但句法分析依然有错误
    所以需要将对应词替换为本来就可以正确分词的内容
    例：泄压部位应能在爆炸作用达到结构最大耐受压强前泄压；
    '''
    words = ltp.pipeline([txt], tasks = ["cws"]).cws[0]
    # print('@@@@', words)
    recover_dic = {}
    for i in range(len(words)):
        if words[i] in replace_dic:
            recover_dic[i] = words[i]
            words[i] = replace_dic[words[i]]
    return ''.join(words), recover_dic

def span_contain(span1, span2):
    '''span1是否包含span2'''
    if span1[0]<=span2[0] and span1[1]>=span2[1]:
        return True
    return False

def del_intra_span(phrase_span):
    '''去掉被其他span包围的span'''
    if not phrase_span: return phrase_span
    merge_span = [phrase_span[-1]]
    for i in range(len(phrase_span)-2, -1, -1):
        cur_span = phrase_span[i]
        if not span_contain(merge_span[-1], cur_span):
        # if (cur_span[0]-merge_span[-1][0])*(merge_span[-1][1]-cur_span[1])<0:
            merge_span.append(cur_span)
    merge_span.reverse()
    return merge_span

def del_inter_span(lst1, lst2):
    '''只保留span1中没有被span2中元素完全覆盖的部分'''
    i, j = 0, 0
    len1, len2 = len(lst1), len(lst2)
    del_idx = []
    while i<len1 and j<len2:
        if span_contain(lst2[j], lst1[i]):
            del_idx.append(i)
            i += 1
            continue
        # if lst1[i][1]<=lst2[j][0]: i+=1
        if lst2[j][1]<=lst1[i][0]: j += 1
        else: i += 1
    lst = [lst1[i] for i in range(len1) if i not in del_idx]
    return lst


'''
node-edge-node
node-edge-tri
tri-edge-tri
'''
class Trip:
    def __init__(self, head, rel, tail):
        '''
        mode: 元素的表达形式
            span表示起止范围，如[2,5]；
            str表示字符串，如'灭火器'
        '''
        self.head = head
        self.rel = rel
        self.tail = tail
        # self.val = '[{}-{}-{}]'.format(
        #     head[1], rel[1], tail[1]
        # )

class Node:
    def __init__(self, start, end):
        self.start = start
        self.end = end

def have_dep(idx, tag, dep, dep_T2H):
    '''判断当前词是否做对应成分'''
    f1 = tag in dep_T2H[idx+1]
    f2 = False
    if dep[idx][2] == 'COO':
        f2 = have_dep(dep[idx][1]-1, tag, dep, dep_T2H)
    return f1 or f2

def get_all_coo(idx, dep_T2H):
    '''按照在序列中出现的顺序得到和当前词并列的所有词'''
    c1 = dep_T2H[idx+1].get('COO', [])
    c2 = []
    for c in c1:
        c2.append(c)
        c2 += get_all_coo(c-1, dep_T2H)
    return c2

In [4]:
import re
seg = list('asdfs（）sda')
seg.index('（')

5

In [5]:
%load_ext autoreload
%autoreload
# %aimport KPR
import KPR
'''
autoreload的意思是自动重新装入，它后面可带参数。
无参：装入所有模块
0：不执行装入命令。
1：只装入所有%aimport 要装的模块。
2：装入所有%aimport不包含的模块。'''


class RuleNER:
    '''
    用规则系统做NER
    基本上所有以名词为中心语的短语都可以当作实体先抽出来
    '''
    def __init__(self, ltp) -> None:
        self.ltp = ltp
    
    def get_spd(self, txt):
        txt, recover_dic = deal_miss_word(txt, replace_dic)
        result = self.ltp.pipeline([txt], tasks = ["cws","dep","pos"])
        seg = result.cws[0]
        pos = result.pos[0]
        dep = result.dep[0]
        for key in recover_dic:
            seg[key] = recover_dic[key]
        dep = list(zip(range(1,1+len(seg)), dep['head'], dep['label']))
        return seg, pos, dep

    def test(self, txt):
        '''测试所有函数'''
        seg, pos, dep = self.get_spd(txt)
        new_txt = self.del_brackets(seg, pos, dep)
        print(new_txt)

    def get_att_head_phrase(self, txt):
        '''得到定中短语'''
        seg, pos, dep = self.get_spd(txt)
        dep_T2H = build_T2H(dep)   # 构建发射字典
        phrase_span = KPR.get_so(seg, pos, dep, dep_T2H)
        phrases = [seg[s[0]-1:s[1]] for s in phrase_span]
        return phrases

    def get_full_trips(self, txt, offset=0):
        '''得到扩展三元组'''
        seg, pos, dep = self.get_spd(txt)
        dep_T2H = build_T2H(dep)   # 构建发射字典
        print('71 self.dep_T2H', dep_T2H)
        so_spans = KPR.get_so(seg, pos, dep, dep_T2H)
        adv_spans = self.get_adv(seg, pos, dep, dep_T2H)
        preda_spans = self.get_preda(seg, pos, dep, dep_T2H)

        idx_so_span = {}
        idx_uncon_so_span = {}
        idx_preda_span = {}
        for span in so_spans:
            idx_so_span.update({s:span for s in range(span[0], span[1]+1)})
        # for span in uncon_so_spans:
        #     idx_uncon_so_span.update({s:span for s in range(span[0], span[1]+1)})
        for span in preda_spans:
            idx_preda_span.update({s:span for s in range(span[0], span[1]+1)})
        head = list(dep_T2H[0].values())[0][0]
        hed_list = [head] + get_all_coo(head-1, dep_T2H)
        ########## 要处理解析结果中没有谓语的情况 ##########
        # 相对密度不小于0.75的可燃气体
        print('132', hed_list)
        hed_spans = [idx_preda_span[i] for i in hed_list]
        hed_spans = del_intra_span(hed_spans)
        print('135', hed_list, hed_spans)
        
        single_trips = self.get_single_trips(
            seg, pos, dep, dep_T2H, hed_spans, idx_preda_span, 
            idx_so_span, idx_uncon_so_span
        )
        
        def shift_offset(spans):
            '''递归得到字符串'''
            spans_shift = []
            for s in spans:
                # print('414', s)
                # if len(s)==3:
                if not s:
                    span_str = None
                elif s[-1]=='tri':
                    span_str = shift_offset(s[:-1])
                    span_str.append('tri')
                elif s[-1]!='node':
                    span_str = shift_offset(s)
                else:
                    span_str = [s[0]+offset, s[1]+offset]
                    span_str.append('node')
                spans_shift.append(span_str)
            return spans_shift
        
        single_trips = [
            shift_offset(single_trip) for single_trip in single_trips
        ]
        # print('154', single_trips)
        return single_trips

    def get_att_head(self, seg, pos, dep, dep_T2H):
        '''
        提取以助词“的”结尾的定语：
            从后往前找，碰到“的”则判断是否为定语
            “的”不在短句最后，RAD的词的覆盖范围
        去掉定语后的句子，定语在句子中的对应部分
        给水厂的设计规模应满足供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求
        '''
        

    def get_adv(self, seg, pos, dep, dep_T2H):
        '''
        抽取状语
        状中结构，从中心语向前找
        介宾结构，从介词向后找
        '''
        phrase_span = []
        for i in range(len(pos)):
            if dep[i][2]=='ADV':
                # 状中结构ADV确定状语结束位置：当温度大于30，湿度大于50时，体育场要打开抽湿器，关闭加热器
                start = find_smallest(dep_T2H, i+1)
                end = dep_T2H[i+1].get('POB', [i+1])[0]
                phrase_span.append([start, end])
            elif dep[i][2]=='POB' and dep[dep[i][1]-1]=='ADV':
                # 介宾短语POB确定状语的结束位置：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
                phrase_span.append([dep[i][1], i+1])
        merge_span = del_intra_span(phrase_span)
        phrases = [seg[s[0]-1:s[1]] for s in merge_span]
        print('adv phrases', phrases)
        return merge_span

    def get_preda(self, seg, pos, dep, dep_T2H):
        '''
        像SO一样抽取所有的谓词，在三元组抽取中，用HED及其并列位置借助字典进行映射
        抽取谓词，没有处理主语从句、定语从句的谓词
        句子的核心词及其并列词，且是动词或介词
        是动词时需要将补语算作谓词的一部分
        介词做谓语：仓库应在居住区外部
        补语结构：仓库应设在居住区外部
        '''
        # 0: {'HED': [20]}
        def is_concat_preda(i, pre_span):
            '''
            谓词合并需要满足三个条件
            f1：连词直接连到上一个span后面
            f2：当前谓词跟上一个span的谓词是并列关系
            f3：当前谓词没有直接相连的FOB或SBV
            样例：
                仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定
                当温度大于或明显等于30，湿度大于50时，体育场要打开抽湿器，关闭加热器
            '''
            # f1 = seg[i-2]=='、' or dep_T2H[i].get('LAD', [-2])[0]==i-1
            # print('245', i, dep[i-1], pre_span, dep[i-1][1]==pre_span[0])
            # f1 = seg[i-2]=='、' or dep_T2H[i].get('LAD', [-2])[0]==pre_span[-1]+1
            # f2 = dep[i-1][1]==pre_span[0]
            # f3 = 'SBV' not in dep_T2H[i] and 'FOB' not in dep_T2H[i]
            # return i>1 and f1 and f2 and f3
            if not pre_span: return False
            f1 = seg[i-1]=='、' or dep_T2H[i+1].get('LAD', [-2])[0]==pre_span[-1]+1
            f1 = f1 or dep[i][0]-dep[i][1]==1                                           # 跟上一个动词连续并列：在建筑防火中贯彻执行国家技术经济政策
            f2 = dep[i][1]==pre_span[0]
            f3 = 'SBV' not in dep_T2H[i+1] and 'FOB' not in dep_T2H[i+1]
            return i>0 and f1 and f2 and f3

        phrase_span = [[]]
        for i in range(len(pos)):
            f1 = False
            for tag in ['SBV','FOB','VOB']:
                if have_dep(i, tag, dep, dep_T2H):
                    f1 = True
                    break
            f2 = pos[i]!='a'
            f3 = pos[i]=='a' and dep_T2H[i+1].get('SBV',[-1])[0]!=i
            if f1 and (f2 or f3):
                start = i + 1
                # 动词带有补语其表意才完整：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
                end = dep_T2H.get(i+1).get('CMP', [i+1])[0]
                if dep[i][2]=='COO' and is_concat_preda(i, phrase_span[-1]):
                    start = phrase_span[-1][0]
                    phrase_span.pop()
                phrase_span.append([start, end])
        phrase_span = phrase_span[1:]

        # phrase_span = []
        # head = dep_T2H[0]['HED'][0]
        # # preda_list = [head] + dep_T2H.get(head,{}).get('COO',[])
        # preda_list = [head] + get_all_coo(head-1, dep_T2H)
        # for i in preda_list:
        #     if pos[i-1] in ['v','p']:
        #         start = i
        #         # 动词带有补语其表意才完整：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
        #         end = dep_T2H.get(i, {}).get('CMP', [i])[0]
        #         if dep[i-1][2]=='COO' and is_concat_preda(i, phrase_span[-1]):
        #             start = phrase_span[-1][0]
        #             phrase_span.pop()
        #         phrase_span.append([start, end])

        phrases = [seg[s[0]-1:s[1]] for s in phrase_span]
        print('preda phrases', phrases)
        # merge_span = del_intra_span(phrase_span)
        # phrases = [seg[s[0]-1:s[1]] for s in merge_span]
        # print('preda merge_span', phrases)
        return phrase_span
    
    def bi_subj(self, preda, seg, pos):
        '''
        处理双主语结构的句子
        看似没有宾语，但实际上是两个主语发生这个动作
        也会出现在定语从句、主语从句中
        满足判定条件则返回双主语构成的三元组
            谓语在句子最后，且要对谓词分类，把谓词出现在句尾的拿出来，标一下分类数据
            连词和谓语之间是一个完整的名词短语
            连词之前的词属于一个名词短语
        样例：
        瓶装液化石油气不应跟其他化学危险物品混放；
        生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触能引起燃烧的场所
        生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触能引起燃烧
        生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触    短语的合并需要打补丁
        '''
        ########## 需要根据谓词类别判断是双主语还是单主语 ##########
        f1 = preda[-1]==len(pos) or pos[preda[-1]]=='wp'        # 谓语在句子最后
        f2 = False
        f3 = False
        for i in range(preda[0]-1, -1, -1):
            if seg[i] in '与和跟': break
        if i>0:
            # 连词和谓语之间是一个完整的名词短语
            txt = ''.join(seg[i+1:preda[0]-1])
            seg1, pos1, dep1 = self.get_spd(txt)
            dep_T2H1 = build_T2H(dep1)   # 构建发射字典
            so_spans1 = KPR.get_so(seg1, pos1, dep1, dep_T2H1)
            if so_spans1 and so_spans1[0][1]-so_spans1[0][0]==preda[0]-i-3:
                so = [i+2, preda[0]-1, 'node']
                f2=True
            # 连词之前的词属于一个名词短语
            txt = ''.join(seg[:i])
            seg1, pos1, dep1 = self.get_spd(txt)
            dep_T2H1 = build_T2H(dep1)   # 构建发射字典
            so_spans1 = KPR.get_so(seg1, pos1, dep1, dep_T2H1)
            if so_spans1 and so_spans1[-1][-1]==i: 
                pre_so = so_spans1[-1] + ['node']
                f3=True
        
        if f1 and f2 and f3: return True, [pre_so], [so]
        return False, [], []

    def get_single_trips(
        self, seg, pos, dep, dep_T2H, hed_spans, idx_preda_span,
        idx_so_span, idx_uncon_so_span
    ):
        '''
        针对每一个中心谓语，提取其三元组及嵌套的结果
        得到简单三元组，解决了宾语从句的表示问题
        根据主谓宾关系得到三元组、二元组
        '''
        
        def get_head_anchor(s):
            '''得到头节点的锚点'''
            anchor = -1
            if 'SBV' in dep_T2H[s]:
                anchor = dep_T2H[s]['SBV'][0]
            elif 'FOB' in dep_T2H[s]:
                anchor = dep_T2H[s]['FOB'][0]
            elif dep[s-1][2] == 'COO':
                anchor = get_head_anchor(dep[s-1][1])
            return anchor
        
        def get_tail_anchor(s):
            '''
            得到尾节点的锚点
            有动宾结构和介宾结构两种可能
            '''
            anchor = -1
            if 'VOB' in dep_T2H[s]:
                anchor = dep_T2H[s]['VOB'][0]
            elif pos[s-1]=='p':
                anchor = dep_T2H[s].get('POB', [-1])[0]
            elif s<len(pos) and pos[s]=='p' and dep[s][1]==s:
                anchor = dep_T2H[s+1].get('POB', [-1])[0]
            return anchor

        def preda_of_clause(anchor):
            '''判断当前词是否为从句的谓词'''
            f1 = pos[anchor-1] in ['v','p']
            f2 = False
            for tag in ['SBV','FOB','VOB']:
                if have_dep(anchor-1, tag, dep, dep_T2H):
                    f2 = True
                    break
            return f1 and f2

        def get_node(anchor):
            '''
            根据anchor得到节点范围
            谓词有COO则直接拆分为多个阶段
            非谓词有COO则将其范围内连续的COO合成一个
            '''
            if anchor == -1:
                return [None]
            elif preda_of_clause(anchor):
                # 当前中心语是宾语从句的谓词，则返回三元组列表
                # anchor_list = [anchor] + get_all_coo(anchor-1, dep_T2H)
                # node_list = []
                # node_list = [get_recur_trip(idx_preda_span[a]) for a in anchor_list]
                # node_list = [node.append('tri') for node in node_list]
                # for a in anchor_list:
                    # node_list += get_recur_trip(idx_preda_span[a])
                    
                # 规范树解析出错，需要分级解析：本规范要预防建筑火灾和减少火灾危害，确保生命财产的安全，并在建筑防火中贯彻执行国家技术经济政策，确保建筑的防火符合安全可靠、经济合理、技术先进、确保质量的要求
                # 不用再管并列，把从句当主句时自会处理并列：当温度大于30，湿度大于50时，我们要保证体育场打开抽湿器，关闭加热器
                start = find_smallest(dep_T2H, anchor)
                end = find_biggest(dep_T2H, anchor)
                txt = ''.join(seg[start-1:end])
                # print('403', start, txt)
                node_list = self.get_full_trips(txt, start-1)
                for n in node_list: n.append('tri')
                # node = get_recur_trip(idx_preda_span[anchor])
                # 有COO则返回多个trip列表，统一格式，都返回node的列表
                # 谓词也学SO搞个idx到span的映射，处理并列的情况
                # return node
                return node_list
            else:
                # start = find_smallest(dep_T2H, anchor)
                # node = Node(start, anchor)
                # node = [start, anchor]
                node = idx_so_span.get(anchor, None)
                if node[-1]!='node': node.append('node')
                return [node]

        def get_recur_trip(preda):
            '''
            得到带有递归节点的三元组
            anchor有COO：甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站
            从句有COO：建筑的防火应符合下列目标要求：保障人身生命和财产安全、人身健康；
            '''
            flag, head, tail = self.bi_subj(preda, seg, pos)

            if not flag:
                anchor = get_head_anchor(preda[0])
                head = get_node(anchor)

                for p in preda:                         # 存在补语加VOB的情况：我们应看完饭、吃完饭、做完饭
                    anchor = get_tail_anchor(p)
                    if anchor!=-1: break
                tail = get_node(anchor)

            # 有多个head或tail则做笛卡尔积
            if preda[-1]!='node': preda.append('node')
            trip_list = []
            '''
            如果head和tail是node则要解析其定语
                有谓语则解析句子，没谓语则当作节点定语
            针对preda解析其状语，句子以句号为分隔
                有谓语则解析句子，没谓语则当节点状语
            可以先不处理定语
            基本节点的格式为：{
                trip: [h, preda, t],
                modi_h: [node, cell],
                modi_t: [node, cell],
                adv: [node, cell]
            }
            '''
            adv_list = []
            for adv_anchor in dep_T2H:
                pass
            for h in head:
                for t in tail:
                    trip_list.append([h,preda,t])
            return trip_list

        trip_spans = []
        print('510', hed_spans)
        for preda in hed_spans:
            '''
            根据谓词判断三元组，借助COO处理省略主语的情况
            头实体：先找SBV、FOB，没有则顺着COO找，没有则用上级条文的主语
            尾实体：找VOB，没有则记占位节点
            仓库的防火要求应根据储存物质的性质和储存物质中可燃物的数量等因素确定，并应符合下列规定
            连词相连的谓词要先合并再拆解
            仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定
            如果anchor处是从句的谓语，则递归执行该函数
            '''
            trip_spans += get_recur_trip(preda)
        
        '''在这里给'''
        print(trip_spans)

        def get_str(spans):
            '''递归得到字符串'''
            spans_str = []
            for s in spans:
                # print('414', s)
                # if len(s)==3:
                if not s:
                    span_str = []
                elif s[-1]=='tri':
                    span_str = get_str(s[:-1])
                elif s[-1]!='node':
                    span_str = get_str(s)
                else:
                    span_str = seg[s[0]-1:s[1]]
                spans_str.append(span_str)
            return spans_str
        
        phrases = [
            get_str(trip_span) for trip_span in trip_spans
        ]
        print('single trip phrases', phrases)
        print(trip_spans)
        return trip_spans

    
    def deal_digestion(self):
        '''
        确定其的指代内容
        做三元组成分，则替换为对应的名词短语
        做修饰语，则替换为名词短语+的
        指代判定依据：前一个名词短语或三元组的头
            所有最基本的头尾节点都要经过指代消解
        做修饰语：陆上消防站应位于易燃易爆危险品场所或设施的常年主导风向的上风或侧风处，其用地边界距离甲、乙类厂房、加油、加气站及易燃易爆危险品储存场所不应小于50m；
        做三元组成分：建筑承重结构应保证其在受到火或高温作用后仍能在设计耐火时间内正常发挥功能；
        远距离指代：设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道
        '''
        
    def deal_attribute():
        '''
        处理开头是介词的，之间、间距、距离
        短语开头是介词：消防站执勤车辆的主出入口距离大型人员密集的公共建筑的主要疏散出口不应小于50m。
        短语结尾是之间、间距、距离：甲、乙类工厂和仓库，可燃气体充装站、供应站和调压站，汽车加油加气站等之间及与其他建筑的间距，应符合消防安全要求；
        短语开头是介词，结尾是间距：易燃易爆危险品库房与在建工程的防火间距不应小于15m，与固定动作作业区不应小于12m，与邻近人员密集区、建筑物相对集中区及其他建筑的间距应符合消防要求；
        省略属性，属性值的头实体应该是属性
        '''

    def split_phrase():
        '''
        分割名词短语
        把部位当作属性节点
        先把逗号分开，再按照一般拆解过程来
        '''

    def get_modi():
        '''
        定语从句，谓词是ATT的尾，向前向后找全覆盖
        中心语用连续ATT和ADV确定范围
        面积大于100平米的消防水泵配电应能在火灾时保持不间断供电，其线路应为专用消防配电线路。
        实体的定语还是在关系节点上，内边上不附着定语或状语
        '''
    
    def get_adv_trip():
        '''
        解析状语从句
        情态动词可能在一个状语之前：水处理药剂必须计量投加。
        '''
    
    def get_modi_trip():
        '''解析定语从句'''

'''
把“压强”换成“压力”，解析完后再换回来
词典缺失词替换
提取带定语的名词短语
提取谓词、状语
提取三元组，状语内的名词短语不参与三元组构建
嵌套提取主语从句、宾语从句中的三元组

名词短语处理：
共指消解，其
处理双头属性，开头是介词的，之间、间距、距离

名词短语解析，提取定语，定语分配给各个并列中心语
定语分为修饰、从句、实体
解析定语从句
中心语分类，哪些是实体，哪些是属性
解析一般实体，实体-包含-实体
解析一般属性，实体-内边-属性
处理省略实体、省略属性的属性，属性值之前必是属性
拆解并列中心语
'''

rule_ner = RuleNER(ltp)
txt = '建筑的选址和总平面布局应符合减小火灾危害，方便灭火救援的要求，并应符合下列规定'
# txt = '生产和储存易燃易爆物品的工厂、仓库等应位于城镇规划区的边缘或相对独立的安全地带；'
# txt = '储罐区的低倍数泡沫灭火系统应符合下列规定：对于非水溶性甲、乙、丙类液体固定顶储罐，应为液上喷射、液下喷射或半液下喷射系统；'
txt = '甲、乙类工厂和仓库，可燃气体充装站、供应站和调压站，汽车加油加气站等之间及和其他建筑的间距，应符合消防安全要求；'
# txt = '消防站执勤车辆的主出入口距离大型人员密集的公共建筑的主要疏散出口不应小于50m。'
'''如果开头词性是p，则有可能表示属性，实体末尾是之间、距离，则要当相对属性来处理'''
txt = '陆上消防站应位于易燃易爆危险品场所或设施的常年主导风向的上风或侧风处，其用地边界距离甲、乙类厂房、加油、加气站及易燃易爆危险品储存场所不应小于50m；'
txt = '建筑内的防火分隔应能在其设计耐火时间内阻止火势与烟气蔓延至其他区域'
txt = '设置泄压设施时，泄压部位应能在爆炸作用达到结构最大耐受压强前泄压，其泄压方向不应朝向人员聚集的场所和人行通道；'
# txt = '除粮食等筒仓外，无法设置泄压设施或泄压面积不符合要求时，相应部位的建筑承重结构和防火分隔结构应满足抗爆要求。'
'''有没有介词会很影响状语的抽取规则'''
txt = '当仓库足够大时，仓库的防火要求应根据储存物质的性质和储存物质中可燃物的数量等因素确定，并应符合下列规定：'
txt = '仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定'
# txt = '仓库不应设在居住区内部'
txt = '保障施工现场消防供水的消防水泵配电应能在火灾时保持不间断供电，其线路应为专用消防配电线路。'
# txt = '戊类仓库的防火要求'
# txt = '建筑承重结构应保证其在受到火或高温作用后仍能在设计耐火时间内正常发挥功能'
txt = '甲、乙类工厂和仓库应设在可燃气体充装站、供应站和调压站、加油站'
txt = '当温度大于或明显等于30，湿度大于50时，体育场要打开抽湿器，关闭加热器'
# txt = '温度大于30，湿度大于50'
# txt = '当温度大于30或高度明显等于30，湿度大于50时，体育场要打开抽湿器，关闭加热器'
# txt = '建筑的防火应符合下列目标要求：保障生命财产安全和身体健康；'
txt = '当温度大于30，湿度大于50时，我们要保证体育场打开抽湿器，关闭加热器'
# txt = '仓库应根据储存物质的性质和储存物质中可燃物的数量等因素确定和规划防火要求，并应符合下列规定'
# txt = '为了预防建筑火灾和减少火灾危害，确保生命财产的安全，并在建筑防火中贯彻执行国家技术经济政策，确保建筑的防火符合安全可靠、经济合理、技术先进、确保质量的要求，依据有关法律、法规，制定本规范。'
# txt = '新建、改建和扩建建筑在规划、设计、施工与使用中的防火技术与措施，应遵守本规范。' # 有两个SBV
txt = '当建筑防火中采用的方法、技术、材料与制品、措施等与本规范的规定不同或有特殊要求时，应根据本规范第2.1节进行合规性判定。'
txt = '当方法与制品等与规定不同或有特殊要求时，应根据本规范第2.1节进行合规性判定。'
# txt = '建筑防火中采用的方法、技术、材料与制品、措施等与本规范的规定不同或有特殊要求'
# txt = '无障碍设施应保证安全和各类人群的方便使用。'
# txt = '本规范适用于新建、扩建、改建的民用与工业建筑中自动喷水灭火系统的设计。'
# txt = '本规范要预防建筑火灾和减少火灾危害，确保生命财产的安全，并在建筑防火中贯彻执行国家技术经济政策，确保建筑的防火符合安全可靠、经济合理、技术先进、确保质量的要求'
# txt = '建筑中散发较空气重的可燃气体、蒸气或有粉尘、纤维爆炸危险的场所或部位应符合下列规定'
# txt = '瓶装液化石油气不应与其他化学危险物品混放；'
# txt = '生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触'
# txt = '新建、改建或拆除建筑、结构、设备或类似活动所用临时电气线路和照明器具、涉及施工所需的易燃和可燃物质的使用与存放以及施工现场的用火、用电、用气应符合消防安全要求。'
# txt = '地下、半地下场所内不应使用或储存液化石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道。'
# txt = '地下、半地下场所内不应使用或储存液化石油气、可燃气体、液体燃料，不应有燃气或可燃液体配送管道。'
# txt = '相对密度不小于0.75的可燃气体'
txt = '施工临时办公与生活用房、发电机房、变配电站、厨房操作间、锅炉房和可燃材料与易燃易爆物品库房，当采用金属夹芯板材时，其芯材的燃烧性能应为A级。'
########## 符合表会被认为是一个词 ##########
txt = '城乡给水工程建设规模的划分应符合表2.0.23的要求'  # 符合替换为满足
# txt = '城乡给水工程建设规模的划分应按照表2.0.23的要求设置'
txt = '给水厂的设计规模应满足供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求'
res = rule_ner.get_spd(txt)
word_pos = list(zip(res[0], res[1], res[2]))
# print(word_pos)
# print(res[2])
# rule_ner.get_full_trips(txt)


#### 括号删除函数

In [6]:
def del_brackets(txt, seg, pos, dep):
    '''
    返回删掉括号中内容的句子
    括号中是修饰后续内容的数值则保留括号里内容
        原料库房与设备间（3）均应有保持良好通风的设备，换气次数应为（8~12）次/h
    '''
    if '（' not in txt:                                 # 原文中没有括号
        return txt
    txt_flag = False
    if '（' not in seg:                                 # 分词错误导致分词结果中没有括号
        seg = txt
        txt_flag = True
    len_s = len(seg)
    del_list = [False] * len_s
    flag = False
    for i in range(len_s-1):
        if seg[i]=='（':
            del_list[i] = True
            if not txt_flag:
                if pos[i+1]=='m' and dep[i+1][1]>i+2:   # 括号中是修饰后续内容的数值则保留括号里内容
                    continue
            flag = True
        elif seg[i]=='）':
            del_list[i] = True
            flag = False
        else:
            del_list[i] = flag
    if seg[-1] in '（）': del_list[-1] = True
    new_seg = [seg[i] for i in range(len_s) if not del_list[i]]
    return ''.join(new_seg)

txts = [
    '原料库房与设备间（3）均应有保持良好通风的设备，换气次数应为（8~12）次/h',
    '大面积的多层地下建筑物（如地下车库、商场、运动场等）',
    '复杂地质条件下的坡上建筑物（包括高边坡）',
    '基坑工程、边坡工程设计时，应根据支护（挡）结构破坏可能产生的后果（危及人的生命、造成经济损失、对社会或环境产生影响等）的严重性，采用不同的安全等级。',
    '支护（挡）结构安全等级的划分应符合表 2.2.4 的规定。',
    '所有建（构）筑物的地基计算均应满足承载力要求；',
    '土和（或）水对建筑材料的腐蚀性；'
]
for txt in txts:
    seg, pos, dep = rule_ner.get_spd(txt)
    new_txt = del_brackets(txt, seg, pos, dep)
    print(new_txt)

原料库房与设备间均应有保持良好通风的设备，换气次数应为8~12次/h
大面积的多层地下建筑物
复杂地质条件下的坡上建筑物
基坑工程、边坡工程设计时，应根据支护结构破坏可能产生的后果的严重性，采用不同的安全等级。
支护结构安全等级的划分应符合表 2.2.4 的规定。
所有建筑物的地基计算均应满足承载力要求；
土和水对建筑材料的腐蚀性；


#### 提取“的”引导的定语

In [7]:
%load_ext autoreload
%autoreload
# %aimport KPR
import KPR

def get_u_att(span, seg, pos, dep):
    '''
    找到一定范围内所有“的”引导的定语
    return:
        [[开始位置，结束位置，定语的牵引点位置，中心语位置]]
    '''
    if not span: span=[1, len(seg)]
    start, end = span[0], span[1]
    span_T2H = build_T2H(dep[start-1:end])
    att_span_list = []
    i = end-1
    while i>=start:
        if seg[i]=='的':
            att_start = find_smallest(span_T2H, dep[i][1])
            if pos[att_start-1]=='wp': att_start+=1
            center = dep[dep[i][1]-1][1]
            att_span_list.append([att_start, i, dep[i][1], center])
            i = att_start - 1
        i -= 1
    # phrases = [[seg[s[0]-1:s[1]], seg[s[2]-1], seg[s[3]-1]] for s in att_span_list]
    # if phrases:
    #     print('u_att phrases', phrases)
    return att_span_list

txts = [
    '原料库房与设备间（3）均应有保持良好通风的设备，换气次数应为（8~12）次/h',
    '基坑工程、边坡工程设计时，应根据支护（挡）结构破坏可能产生的后果（危及人的生命、造成经济损失、对社会或环境产生影响等）的严重性，采用不同的安全等级。',
    '支护（挡）结构安全等级的划分应符合表 2.2.4 的规定。',
    '所有建（构）筑物的地基计算均应满足承载力要求；',
    '当温度大于30，湿度大于50时，我们要保证体育场打开抽湿器，关闭加热器',
    '给水厂的设计规模应满足供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求',
    '保养厂应能承担营运车辆的高级保养任务及相应的配件加工、修制和修车材料、燃料的储存、发放等',
    '控制中心应具备行车调度、电力调度、环境与设备调度、防灾指挥、客运管理、乘客信息管理、设备维修及信息管理等运营调度和指挥功能，并应对城市轨道交通系统运营的全过程进行集中监控和管理',
    '给水厂应对制水生产中的主要设施、设备制定和实施巡查维护保养制度，应对主要工艺运行情况及其运行中的动态技术参数，制定和实施质量控制点检验制度',
    '建筑无法设置泄压设施或泄压面积不符合要求时，建筑中存在可燃气体、蒸气、粉尘或纤维爆炸危险的部位的建筑承重结构应满足抗爆要求。',
    '建筑中存在可燃气体、蒸气、粉尘或纤维爆炸危险的部位的建筑承重结构应满足抗爆要求。',
]
# 去除括号中的文本，抽取定中短语中“的”引导的定语
for txt in txts:
    seg, pos, dep = rule_ner.get_spd(txt)
    new_txt = del_brackets(txt, seg, pos, dep)
    if new_txt:
        seg, pos, dep = rule_ner.get_spd(new_txt)
    else:
        new_txt = txt
    dep_T2H = build_T2H(dep)
    so_spans = KPR.get_so(seg, pos, dep, dep_T2H)
    # print('txt', new_txt)
    for span in so_spans:
        print(seg[span[0]-1:span[1]])
        get_u_att(span, seg, pos, dep)
    # print(new_txt)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
131 [[1, 5], [8, 12], [14, 15], [18, 19]]
['原料', '库房', '与', '设备', '间']


TypeError: unsupported operand type(s) for -: 'str' and 'int'

In [8]:
import jieba
from gensim.models import KeyedVectors
from model_TS import TxtSim
word_vec_tenc = 'D:\Download\ArchData\data\PreData\\arch-zh-d200-tencent-tp200.txt'
wv_from_text_word = KeyedVectors.load_word2vec_format(
    word_vec_tenc, binary=False, no_header=False)

from text2vec import SBert
sbert = SBert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2022-12-14 20:12:03.895 | DEBUG    | text2vec.sentence_model:__init__:74 - Use device: cpu


In [32]:
%load_ext autoreload
%autoreload
# %aimport KPR
import KPR
word_sim = TxtSim(wv_from_text_word, jieba, sbert)
k_spliter = KPR.WordSpliter(ltp, word_sim)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 并列部分的分解

In [48]:
# 确定这一整段都是并列的实体或是含并列的定语后执行
'''
只有“的”引导的定语和原生紧邻coo存在分配现象
放在条文预处理：
    儿童活动场所、老年人照料设施中的老年人活动场所、医疗建筑中的治疗室和病房、教学建筑中的教学用房位于走道尽端时，其疏散门不应少于2个
    地下、半地下场所内不应使用或储存液化石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道
0.连续并列合并（全文范围内的合并，包括动词并列）；去掉纯短vatt：父节点只允许有wp和att，且是att中第一个，att向外只允许有wp，且父节点不是别人的att
    儿童活动场所、老年人照料设施中的老年人活动场所、医疗建筑中的治疗室、教学建筑中的教学用房位于走道尽端时，其疏散门不应少于2个
    地下、半地下场所内不应使用石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道
放在给定span的条文蒸馏
0.去掉无用的等；去掉“的”引导的孤立定语，定语前没有连词非连词定语、定语是从句或定语由方位名词引导
    儿童活动场所、老年人活动场所、治疗室、教学用房位于走道尽端时，其疏散门不应少于2个
1.去掉短vatt：父节点只允许有wp和att，且att向外只允许有wp，连续att关系（最近的要是v）、都是短att且相邻（最近的要是v）
    场所、场所、治疗室、用房位于走道尽端时，其疏散门不应少于2个
2.连续合并，必须是在原句中前面没有修饰语的/拆开时若对应单词有自己的修饰则不会继承第一个的修饰
    场所位于走道尽端时，其疏散门不应少于2个

    对出水水质产生影响的化学药剂的加注设备应配置备用设备。
    保养厂应能承担营运车辆的高级保养任务及相应的配件加工、修制和修车材料、燃料的储存、发放等
    儿童活动场所、老年人照料设施中的老年人活动场所、医疗建筑中的治疗室和病房、教学建筑中的教学用房位于走道尽端时，其疏散门不应少于2个
    给水厂的设计规模应满足供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求
    地下、半地下场所内不应使用或储存液化石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道。
0.连续并列合并（全文范围内的合并，包括动词并列），去掉短vatt，去掉无用的等：父节点只允许有wp和att，且att向外只允许有wp
    对水质产生影响的化学药剂的加注设备应配置设备。
    保养厂应能承担车辆的高级保养任务及相应的配件加工、修制和修车材料、燃料的储存
    活动场所、设施中的老年人活动场所、建筑中的治疗室、建筑中的教学用房位于走道尽端时，其疏散门不应少于2个
    给水厂的设计规模应满足范围规定内最高日的用水量、企业用水量、道路和绿地用水量、管网漏损水量及未预见用水量的要求
    地下、半地下场所内不应使用或储存石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道。
1.去掉“的”引导的非连词定语、定语从句和由方位名词引导的直接去除
    对水质产生影响的化学药剂的设备应配置设备。
    保养厂应能承担高级保养任务及相应的配件加工、修制和修车材料、燃料的储存
    活动场所、老年人活动场所、治疗室、教学用房位于走道尽端时，其疏散门不应少于2个
    给水厂的设计规模应满足用水量、企业用水量、道路和绿地用水量、管网漏损水量及未预见用水量的要求
    地下、半地下场所内不应使用或储存石油气、可燃气体、液体燃料，不应有相应的燃气或可燃液体配送管道。
2.连续合并，必须是在原句中前面没有修饰语的/拆开时若对应单词有自己的修饰则不会继承第一个的修饰
    对水质产生影响的化学药剂的设备应配置设备。
    保养厂应能承担高级保养任务及相应的配件加工和修车材料、燃料的储存
    活动场所、老年人活动场所、教学用房位于走道尽端时，其疏散门不应少于2个
    给水厂的设计规模应满足用水量、企业用水量、道路用水量、管网漏损水量及未预见用水量的要求
    地下、半地下场所内不应使用或储存石油气、可燃气体、液体燃料，不应有相应的燃气或可燃液体配送管道。
3.去掉短vatt

相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料
可燃气体、液体燃料

存在连词或顿号时才执行下述步骤
    给水厂的设计规模应满足供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求
    保养厂应能承担营运车辆的高级保养任务及相应的配件加工、修制和修车材料、燃料的储存、发放等
    控制中心应具备行车调度、电力调度、环境与设备调度、防灾指挥、客运管理、乘客信息管理、设备维修及信息管理等运营调度和指挥功能，并应对城市轨道交通系统运营的全过程进行集中监控和管理
    给水厂应对制水生产中的主要设施、设备制定和实施巡查维护保养制度，应对主要工艺运行情况及其运行中的动态技术参数，制定和实施质量控制点检验制度
    建筑中存在可燃气体、蒸气、粉尘或纤维爆炸危险的部位的建筑承重结构应满足抗爆要求。
1.去掉“的”引导的长定语、“的”引导的动词短定语-所有定语都是动词，去掉无用的“等”，连续并列合并，去掉“ATT”引导的动词短定语-只有连续挨着中心词的定语
1.去掉“的”引导的长定语，去掉无用的“等”，连续并列合并，去掉“ATT”引导的动词短定语-只有连续挨着中心词的定语
1.去掉“的”引导的孤立定语，去掉无用的“等”，去掉“ATT”引导短定语-没有出边且父节点不是定语，连续并列合并
    给水厂的设计规模应满足综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求
    保养厂应能承担车辆的高级保养任务及配件加工、修制和修车材料、燃料的储存
    控制中心应具备行车调度、电力调度、环境与设备调度、防灾指挥、客运管理、乘客信息管理、设备维修及信息管理等运营调度功能，并应对全过程进行集中监控
    建筑中存在可燃气体或纤维爆炸危险的部位的建筑承重结构应满足抗爆要求。
2.去掉“ATT”引导的短定语-不是状语且连续，连续并列合并(直接COO且都是单词)，去掉短定语(只有一个单词定语)
    给水厂的设计规模应满足用水量、工业企业用水量、道路和绿地用水量、管网漏损水量及未预见用水量的要求
    保养厂应能承担高级保养任务及配件加工、修制和材料、燃料的储存
    控制中心应具备行车调度、电力调度、环境与设备调度、指挥、客运管理、乘客信息管理、设备维修及信息管理等运营调度功能，并应对全过程进行集中监控
    建筑中存在气体或纤维爆炸危险的部位的建筑承重结构应满足抗爆要求。
3.连续并列合并
    给水厂的设计规模应满足用水量、工业企业用水量、道路用水量、管网漏损水量及未预见用水量的要求
    保养厂应能承担高级保养任务及配件加工和材料的储存
    控制中心应具备行车调度、电力调度、环境与设备调度、指挥、客运管理、乘客信息管理、设备维修及信息管理等运营调度功能，并应对全过程进行集中监控
    建筑中存在气体爆炸危险的部位的建筑承重结构应满足抗爆要求。
4.用现有方法做并列解析

底层得到树状解析结果后，往上找每个中心语在上层对应的词语
    有vatt则则给到单个中心语
    有coo则记在当前中心语的名下
    有uatt：
        只有一个且在同一层次的句首，则分配给该层次的每个节点
        多个或不在句首，则只修饰对应节点

'''

%load_ext autoreload
%autoreload
# %aimport KPR
import KPR

def have_dep(idx, tag, dep, dep_T2H):
    '''判断当前词是否做对应成分'''
    f1 = tag in dep_T2H[idx+1]
    f2 = False
    if dep[idx][2] == 'COO':
        f2 = have_dep(dep[idx][1]-1, tag, dep, dep_T2H)
    if f1 and tag == 'FOB':
        fob = dep_T2H[idx+1][tag][0]
        f3 = fob<idx
        return (f1 and f3) or f2
    else:
        return f1 or f2

def is_preda_of_clause(anchor, dep, dep_T2H):
    '''判断当前词是否为从句的谓词'''
    f1 = pos[anchor-1] in ['v','p']
    f2 = False
    for tag in ['SBV','FOB','VOB']:
        if have_dep(anchor-1, tag, dep, dep_T2H):
            f2 = True
            break
    return f1 and f2

def is_simple_uatt(uatt_span, dep):
    '''
    判断是否为简单uatt
    长度不大于3 且 (“的”的上级节点是左边第一个 或 内部关系都是COO、ATT)
    需要COO判断的情况：修车材料、做饭燃料的储存、发放等
    '''
    u_s = uatt_span[0]
    u_e = uatt_span[1]
    f1 = u_e-u_s<=2
    if not f1: return f1
    f2 = dep[u_e][1]==u_e
    if f2: return f1 and f2
    f3 = all([dep[i][2] in ['COO','ATT'] for i in range(u_s-1, u_e)])
    return f1 and (f2 or f3)

def is_isolated_uatt(uatt_span, seg, pos, dep, dep_T2H):
    '''
    判断是否为孤立uatt
    定语的前一个词既不是连词也不是顿号
    当前定语由方位名词引导
    当前定语是定语从句
    '''
    u_s, u_e = uatt_span[0], uatt_span[1]
    f1 = u_s>1 and seg[u_s-2]!='、' and pos[u_s-2]!='c'
    f2 = pos[u_e-1]=='nd'
    f3 = is_preda_of_clause(uatt_span[2], dep, dep_T2H)
    return f1 or f2 or f3

def is_pure_short_att(idx, dep, dep_T2H):
    '''
    判断是否为短定语
    中心语只有这一个定语且和中心语相邻
    去掉“ATT”引导的动词短定语-只有连续挨着中心词的定语
    f1: 跟父节点是修饰关系且父节点是下一个词
    f2: 父节点只有这么一个修饰语
    f3: 除了标点没有向外的箭头
    f4: 父节点不是状语
    '''
    f1 = dep[idx][2]=='ATT' and dep[idx][1]==idx+2
    if not f1: return f1
    f2 = len(dep_T2H[dep[idx][1]].get('ATT', []))==1
    if not f2: return f1 and f2
    f3 = not dep_T2H[idx+1] or list(dep_T2H[idx+1].keys())==['WP']
    if not f3: return f1 and f2 and f3
    f4 = dep[dep[idx][1]-1][2]!='ADV'
    return f1 and f2 and f3 and f4

def get_general_short_vatt(idx, pos, dep, dep_T2H):
    '''
    以每个词为中心向前找svatt的范围
    中心语只有这一个定语且和中心语相邻
    去掉“ATT”引导的动词短定语-只有连续挨着中心词的定语
    f1: 不是状语
    f2: 前一个词是v且是att
    f3: 所有att都只有向前的att
    f4: att的边界连续
    '''
    f1 = dep[idx][2]!='ADV'
    if not f1: return []
    f2 = idx>0 and pos[idx-1]=='v' and dep[idx-1]==[idx,idx+1,'ATT']
    if not f2: return []
    att_list = dep_T2H[idx+1].get('ATT', [])
    f3_list = []
    att_spans = []
    legal_keys = [{}, {'WP'}, {'ATT'}, {'WP','ATT'}]
    for j in att_list:
        keys = set(list(dep_T2H[j].keys()))
        if not keys in legal_keys:
            f3_list.apend(False)
            break
        start = j
        for k in range(j-2,-1,-1):
            keys = set(list(dep_T2H[j].keys()))
            flag = dep[k]==[k+1,k+2,'ATT'] and keys in legal_keys
            if not flag:
                f3_list.append(False)
                break
            if flag and keys in legal_keys[:2]:
                f3_list.append(True)
                start = j + 1
                break
        if not f3_list[-1]: break
        att_spans.append([start,j])
    if not all(f3_list): return []
    if not att_spans: return []
    if len(att_spans)==1: return att_spans[0]
    for j in range(len(att_spans)-1):
        if att_spans[j+1][0]-att_spans[j][1]>1: return []
    return [att_spans[0][0], att_spans[-1][-1]]

def get_continum_short_coord_span(span, seg, dep_T2H):
    '''
    找到连续短并列结构的范围
    都是单个词，间隔只有顿号或连词
    间隔必须是同一个，连续顿号或者连续某个连词
    '''
    if not span: span = [1, len(seg)]
    i = span[0]-1
    short_coo_spans = []
    while i<span[1]:
        coos = dep_T2H[i+1].get('COO', [])              # 并列的内容都会标在第一个的后面
        p = i + 1
        if coos: 
            conj = seg[i+1]
            # print(i, seg[i], dep_T2H[i+1])
            for c in coos:
                # print('99', c, seg[c-1])
                if c-p==2 and seg[p]==conj:             # 中间只隔一个词且该词是第一个连词
                    p = c
                else:
                    break
            if p>i+1:                                   # 只记录连续短并列结构
                short_coo_spans.append([i+1, p])
                i = c
            else:
                i = c-1
        else:
            i += 1
    return short_coo_spans

'''
只有“的”引导的定语和原生紧邻coo存在分配现象
放在条文预处理：
    儿童活动场所、老年人照料设施中的老年人活动场所、医疗建筑中的治疗室和病房、教学建筑中的教学用房位于走道尽端时，其疏散门不应少于2个
    地下、半地下场所内不应使用或储存液化石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道
0.连续并列动词合并（全文范围内的合并，包括动词并列）；去掉纯短vatt：父节点只允许有wp和att，且是att中第一个，att向外只允许有wp，且父节点不是别人的att
    儿童活动场所、老年人照料设施中的老年人活动场所、医疗建筑中的治疗室、教学建筑中的教学用房位于走道尽端时，其疏散门不应少于2个
    地下、半地下场所内不应使用石油气、相对密度不小于0.75的可燃气体、闪点低于60℃的液体燃料，不应有相应的燃气或可燃液体配送管道
放在给定span的条文蒸馏
1.去掉无用的等；去掉“的”引导的孤立定语，定语前没有连词非连词定语、定语是从句或定语由方位名词引导
    儿童活动场所、老年人活动场所、治疗室、教学用房位于走道尽端时，其疏散门不应少于2个
2.去掉短vatt：父节点只允许有wp和att，且att向外只允许有wp，连续att关系（最近的要是v）、都是短att且相邻（最近的要是v）
    场所、场所、治疗室、用房位于走道尽端时，其疏散门不应少于2个
3.连续合并，必须是在原句中前面没有修饰语的/拆开时若对应单词有自己的修饰则不会继承第一个的修饰
    场所位于走道尽端时，其疏散门不应少于2个
'''

def show_spd(seg, pos, dep, tag='246'):
    word_pos = list(zip(seg, pos, dep))
    print(f'{tag} word_pos', word_pos)

def distill_0(span, seg, pos, dep, dep_T2H, del_indic, ellip_dic):
    '''
    连续并列合并
    去掉纯短vatt
    '''
    if not span: span=[1,len(seg)]
    # 合并连续短并列
    short_coo_spans = get_continum_short_coord_span(span, seg, dep_T2H)
    for coo_span in short_coo_spans:
        start, end = coo_span
        # print('253', span, coo_span)
        del_indic[start:end] = [True]*(end-start)
        ellip_dic[start]['scoo'] = [start+1, end]
    # 去掉纯短vatt
    for i in range(span[0]-1, span[1]):
        if not del_indic[i]:
            flag = is_pure_short_att(i, dep, dep_T2H) and pos[i]=='v'
            del_indic[i] = flag
            if flag:
                # print('268 短定语', i, flag, seg[i])
                ellip_dic[i+2]['svatt'] = [i+1, i+1]

def distill_1(span, seg, pos, dep, dep_T2H, del_indic, ellip_dic):
    ''''''
    # 去掉“的”引导的孤立定语
    uatt_span_list = get_u_att(span, seg, pos, dep)
    # if uatt_span_list:
    #     print('有 uatt', seg[span[0]-1:span[1]])
    for uatt_span in uatt_span_list:
        start = uatt_span[0]
        end = uatt_span[1]
        flag = is_isolated_uatt(uatt_span, seg, pos, dep, dep_T2H)
        if flag: 
            print('275 孤立定语', flag, seg[start-1:end])
            del_indic[start-1:end+1] = [True]*(end-start+2)
            ellip_dic[end+2]['uatt'] = [start, end]
    # 去掉无用的“等”
    if seg[span[-1]-1]=='等':
        del_indic[span[-1]-1] = True
        print('138 无用的等', del_indic[span[-1]-1], seg[span[-1]-1])

def distill_2(span, seg, pos, dep, dep_T2H, del_indic, ellip_dic):
    '''
    去掉广义短vatt
    以每个词自己为坐标原点向前找svatt的范围
    '''
    for i in range(span[0]-1, span[1]):
        gsvatt_span = get_general_short_vatt(i, pos, dep, dep_T2H)
        if gsvatt_span:
            start, end = gsvatt_span
            del_indic[start-1:end] = [True]*(end-start+1)
            print('293 短定语', i, seg[start-1:end])
            ellip_dic[i]['svatt'] = gsvatt_span

def distill_3(span, seg, pos, dep, dep_T2H, del_indic, ellip_dic):
    '''
    连续并列合并
    '''
    # 合并连续短并列
    short_coo_spans = get_continum_short_coord_span(span, seg, dep_T2H)
    for coo_span in short_coo_spans:
        start, end = coo_span
        del_indic[start:end] = [True]*(end-start)
        ellip_dic[start]['scoo'] = [start+1, end]

def exec_dis_func(txt, idx, inter_cache):
    '''
    在句子的语境下分析一个片段内的并列情况
    去掉“的”引导的孤立定语
    去掉“ATT”引导短定语-没有出边且父节点不是定语
    去掉无用的“等”
    连续并列合并
    '''
    seg, pos, dep = get_spd(txt)
    dep_T2H = build_T2H(dep)
    so_spans = KPR.get_so(seg, pos, dep, dep_T2H)
    del_indic = [False] * len(seg)
    ellip_dic = defaultdict(dict)
    distill_func = globals()[f'distill_{idx}']
    for span in so_spans:
        distill_func(span, seg, pos, dep, dep_T2H, del_indic, ellip_dic)
    new_txt = [seg[i] for i in range(len(seg)) if not del_indic[i]]
    new_txt = ''.join(new_txt)
    inter_cache[f'distill_{idx}'] = {
        'seg':seg, 'del_indic':del_indic, 'ellip_dic':ellip_dic
    }
    return new_txt

def get_spd(txt):
    txt, recover_dic = deal_miss_word(txt, replace_dic)
    # print('333', txt)
    result = ltp.pipeline([txt], tasks = ["cws","dep","pos"])
    seg = result.cws[0]
    pos = result.pos[0]
    dep = result.dep[0]
    for key in recover_dic:
        seg[key] = recover_dic[key]
    dep = list(zip(range(1,1+len(seg)), dep['head'], dep['label']))
    return seg, pos, dep

txts = [
    # '原料库房与设备间（3）均应有保持良好通风的设备，换气次数应为（8~12）次/h',
    # '基坑工程、边坡工程设计时，应根据支护（挡）结构破坏可能产生的后果（危及人的生命、造成经济损失、对社会或环境产生影响等）的严重性，采用不同的安全等级。',
    # '支护（挡）结构安全等级的划分应符合表 2.2.4 的规定。',
    # '所有建（构）筑物的地基计算均应满足承载力要求；',
    # '当温度大于30，湿度大于50时，我们要保证体育场打开抽湿器，关闭加热器',
    # '给水厂的设计规模应满足供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求',
    # '保养厂应能承担营运车辆的高级保养任务及相应的配件加工、修制和修车材料、燃料的储存、发放等',
    '保养厂应能承担高级保养任务及相应的配件加工、修制和修车材料、燃料的储存',
    # '保养厂应能承担高级保养任务及相应的配件加工、修制和修车材料、燃料的储存',
    # '控制中心应具备行车调度、电力调度、环境与设备调度、防灾指挥、客运管理、乘客信息管理、设备维修及信息管理等运营调度和指挥功能，并应对城市轨道交通系统运营的全过程进行集中监控和管理',
    # '给水厂应对制水生产中的主要设施、设备制定和实施巡查维护保养制度，应对主要工艺运行情况及其运行中的动态技术参数，制定和实施质量控制点检验制度',
    # '建筑无法设置泄压设施或泄压面积不符合要求时，建筑中存在可燃气体、蒸气、粉尘或纤维爆炸危险的部位的建筑承重结构应满足抗爆要求。',
    # '建筑中存在可燃气体、蒸气、粉尘或纤维爆炸危险的部位的建筑承重结构应满足抗爆要求。',
]


%load_ext autoreload
%autoreload
# %aimport KPR
import KPR
word_sim = TxtSim(wv_from_text_word, jieba, sbert)
k_spliter = KPR.WordSpliter(ltp, word_sim)

for txt in txts:
    inter_cache = {}
    # 文本预处理
    seg, pos, dep = get_spd(txt)
    new_txt = del_brackets(txt, seg, pos, dep)
    if new_txt != txt:
        seg, pos, dep = get_spd(new_txt)
    print('368', new_txt)
    dep_T2H = build_T2H(dep)
    del_indic = [False] * len(seg)
    ellip_dic = defaultdict(dict)
    distill_0(None, seg, pos, dep, dep_T2H, del_indic, ellip_dic)
    new_txt = [seg[i] for i in range(len(seg)) if not del_indic[i]]
    new_txt = ''.join(new_txt)
    inter_cache['distill_0'] = {
        'seg':seg, 'del_indic':del_indic, 'ellip_dic':ellip_dic
    }
    
    # 渐进三步走
    for idx in [1,2,3]:
        print('380', new_txt)
        new_txt = exec_dis_func(new_txt, idx, inter_cache)
    print('382', new_txt)
    print('393 inter_cache', inter_cache)

    seg, pos, dep = rule_ner.get_spd(new_txt)
    dep_T2H = build_T2H(dep)
    so_spans = KPR.get_so(seg, pos, dep, dep_T2H)
    show_spd(seg, pos, dep, tag='401')
    for span in so_spans:
        txt = ''.join(seg[span[0]-1:span[1]])
        ent_span, ent_list, incl_tri = k_spliter.split_ent(txt,[],[],[])
    print('405', ent_span)
    print('406', ent_list)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
368 保养厂应能承担高级保养任务及相应的配件加工、修制和修车材料、燃料的储存
380 保养厂应能承担高级保养任务及相应的配件加工和材料、燃料的储存
131 [[1, 1], [5, 18]]
380 保养厂应能承担高级保养任务及相应的配件加工和材料、燃料的储存
131 [[1, 1], [5, 18]]
380 保养厂应能承担高级保养任务及相应的配件加工和材料、燃料的储存
131 [[1, 1], [5, 18]]
382 保养厂应能承担高级保养任务及相应的配件加工和材料的储存
393 inter_cache {'distill_0': {'seg': ['保养厂', '应', '能', '承担', '高级', '保养', '任务', '及', '相应', '的', '配件', '加工', '、', '修制', '和', '修车', '材料', '、', '燃料', '的', '储存'], 'del_indic': [False, False, False, False, False, False, False, False, False, False, False, False, True, True, False, True, False, False, False, False, False], 'ellip_dic': defaultdict(<class 'dict'>, {12: {'scoo': [13, 14]}, 17: {'svatt': [16, 16]}})}, 'distill_1': {'seg': ['保养厂', '应', '能', '承担', '高级', '保养', '任务', '及', '相应', '的', '配件', '加工', '和', '材料', '、', '燃料', '的', '储存'], 'del_indic': [False, False, False, False, Fa

In [ ]:
import jieba
from gensim.models import KeyedVectors
from model_TS import TxtSim
word_vec_tenc = 'D:\Download\ArchData\data\PreData\\arch-zh-d200-tencent-tp200.txt'
wv_from_text_word = KeyedVectors.load_word2vec_format(
    word_vec_tenc, binary=False, no_header=False)

from text2vec import SBert
sbert = SBert()

In [37]:
238+176

414

In [ ]:
%load_ext autoreload
%autoreload
import model_Spliter
word_sim = TxtSim(wv_from_text_word, jieba, sbert)
WordSpliter = model_Spliter.WordSpliter
txts = [
    '非水溶性液体外浮顶储罐、内浮顶储罐、直径大于18m的固定顶储罐及水溶性甲、乙、丙类液体立式储罐',
    '高架仓库或高层仓库',
    '设备机房、电梯机房、水箱间、天线等突出物',
    '国道、省道等干线公路及快速路等道路',
    '儿童活动场所、老年人照料设施中的老年人活动场所、医疗建筑中的治疗室和病房、教学建筑中的教学用房',
    '医疗建筑中的治疗室和病房',
    '地下室的底板、外墙以及上部有覆土的地下室顶板',
    '生产过程中散发的可燃气体、蒸气、粉尘或纤维与供暖管道、散热器表面接触能引起燃烧的场所',
    '国道、省道等干线公路及快速路等道路',
    '配件加工、修制和修车材料、燃料的储存、发放',
    '乙、丙、丁、戊类仓库、民用建筑',
    '入侵和紧急报警系统、视频监控系统、出入口控制系统、停车库（场）安全管理系统',
    '客运管理、乘客信息管理、设备维修及信息管理等运营调度和指挥功能',
    '修车材料、燃料的储存、发放',
    '住宅建筑内的汽车库、锅炉房和建筑中的下列场所',
]
txts = [
    '供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求',
    '综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求',
    '综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量'
]
ent_spliter = WordSpliter(ltp, word_sim)
for txt in txts:
    print('261',txt)
    a,b,c=ent_spliter.split_ent(txt,[],[],[])
    print(a,b)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
261 供水范围规定年限内最高日的综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求
[['', '供水范围规定']] []
261 综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的要求
[['综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量的', '要求']] [['', '综合']]
261 综合生活用水量、工业企业用水量、浇洒道路和绿地用水量、管网漏损水量及未预见用水量
[['', '综合']] []


In [ ]:
txt_a = ['综合生活用水量','工业企业用水量','浇洒道路','绿地用水量','管网漏损水量','未预见用水量']
txt_a = ['地下水质量标准中Ⅰ', 'II类']
txt_a = ['非水溶性液体外浮顶储罐','内浮顶储罐','直径大于18m的固定顶储罐','水溶性甲','乙','丙类液体立式储罐']
txt_a = ['场所','场所','治疗室','病房']
'配件加工、修制和修车材料、燃料'
txt_a = ['加工','修制','材料','燃料']
txt_a = ['甲','乙','丙']
txt_a = ['I','II','类']
txt_a = ['底板','外墙']
word_sim.cos_sim(txt_a, txt_a)

array([[0.9999999 , 0.5710485 ],
       [0.5710485 , 0.99999994]], dtype=float32)

In [ ]:
import os
from utils import dic_read_csv

def get_all_uatt_in_txt(txt):
    seg, pos, dep = rule_ner.get_spd(txt)
    new_txt = del_brackets(txt, seg, pos, dep)
    if new_txt:
        seg, pos, dep = rule_ner.get_spd(new_txt)
    else:
        new_txt = txt
    uatt_spans = get_u_att([1,len(seg)], seg, pos, dep)
    return uatt_spans

csv_path = 'D:\Download\ArchData\data\Corpus\csv_'
csv_list = os.listdir(csv_path)
all_ahp = []
all_uatts = []
for name in csv_list:
    if '1城乡' not in name: continue
    file_name = os.path.join(csv_path, name)
    file_dic = dic_read_csv(file_name)
    idx_list = file_dic['编号']
    txt_list = file_dic['原文']
    for i in range(len(idx_list)):
        if idx_list[i] and idx_list[i][0] not in '表注':
            # ahp = rule_ner.get_att_head_phrase(txt_list[i])
            # all_ahp += ahp
            txt = txt_list[i]
            seg, pos, dep = rule_ner.get_spd(txt)
            new_txt = del_brackets(txt, seg, pos, dep)
            if new_txt:
                seg, pos, dep = rule_ner.get_spd(new_txt)
            else:
                new_txt = txt
            uatt_spans = get_u_att([1,len(seg)], seg, pos, dep)
            flag = False
            for d in dep:
                if d[2]=='FOB' and d[1]-d[0]==1: 
                    continue
                    flag = True
                    print('38', ''.join(seg[d[0]-1:d[1]]))
                elif d[2]=='FOB':
                    flag = True
                    print('38', ''.join(seg[d[0]-1:d[1]]))
            if flag: print(txt)
            # print(uatt_spans)
            uatts = [''.join(seg[s[0]-1:s[1]]) for s in uatt_spans]
            all_uatts += uatts
# print(all_ahp)
# print(all_uatts)

38 类别应划为
城乡给水工程主要设施的抗震设防类别应划为重点设防类。
38 检测应按国家规定的标准检验方法执行
水质检测应按国家规定的标准检验方法执行。
38 用水量应根据生产工艺要求确定
工业企业用水量应根据生产工艺要求确定，不得超过国家规定的取用水定额限值。
38 高程也应按江心式、岸边式取水泵房的防洪标准设计
水库取水构筑物的防洪标准应与水库大坝等主要建筑物的防洪标准相同，并应采用设计和校核两级标准。岸上取水泵房采用开放式前池和吸水井（进水池）时，井（池）顶高程也应按江心式、岸边式取水泵房的防洪标准设计。
38 位置应根据给水系统的布局确定
城乡给水厂的位置应根据给水系统的布局确定。
38 泥水、浮渣、废水和废液均应进行
水处理过程中产生的泥水、浮渣、废水和废液均应进行处理处置，严禁擅自直接排入环境水体。
38 参数应按事故工况计算
水处理构筑物及连接管渠的设计参数应按事故工况计算校核。
38 药剂必须计量投加
水处理药剂必须计量投加。
38 合格方可使用
化验室所用的计量分析仪器必须定期进行计量检定，经检定合格方可使用。计量分析仪器在日常使用过程中应定期进行校准和维护。
38 房间应相互隔
各个房间应相互隔开，室内应互不连通；
38 
二氧化氯发生与投加设备间应配备二氧化氯泄漏的低、高检测极限检测仪和报警设施，室内应设喷淋装置。
38 水锤综合防护
长距离管道输水系统的选择应在输水线路、输水方式、管材、管径等方面进行技术、经济比较和安全论证，并应对管道系统进行水力过渡过程分析，采取水锤综合防护措施。
38 管线综合布置
城乡给水管道的平面布置和竖向位置，应保证供水安全，与建（构）筑物及其他管线的安全防护距离应符合国家规定的管线综合布置的要求。
